In [185]:
import operator as op
from math import radians, pi
from IPython.display import display, Math

import forallpeople as si
si.environment("structural", top_level=True)
N, kN, m, mm, Pa, kPa, MPa, GPa = (N, kN, m, mm, Pa, kPa, MPa, GPa) # type: ignore
kNm = kN*m
deg = radians(1)

DELIM = {'+', '-', '/', '@', '%', '^', '(', ')', '|', ' ', '\\', '{', '}', '_'}
CLOSING = {"(":")", "[":"]", "{":"}"}


In [186]:
# NOTE: comparisons (lt, gt), boolean/bitwise operators (not, xor), series operators (concat, count), and function operators (attrgetter, itemgetter) are not implemented
# NOTE: floor and ceiling functions are not implemented
OPS = {'+':op.add, "-":op.sub, "/":op.truediv, "÷":op.truediv, "*":op.mul, "×":op.mul, "⋅":op.mul, "@":op.matmul, "%":op.mod, "^":op.pow}
UNARY_OPS = {op.abs:("|", "|"), op.neg:("(-", ")"), op.pos:("(", ")")}
BINARY_OPS = {op.add:'+', op.sub:"-", op.truediv:"/", op.mul:"×", op.matmul:"@", op.mod:"%", op.pow:"^"}
# stores an expression
class fn:
	def __init__(self, op, t1=None, t2=None):
		self.op = op
		self.t1 = t1
		self.t2 = t2
		return
	def eval(self, values):
#		print(f"t1 = {self.t1} ({type(self.t1)})")
#		print(f"t2 = {self.t2} ({type(self.t2)})")
		# determine value of t1
		if type(self.t1) == int or type(self.t1) == float:
			v1 = self.t1 
		elif type(self.t1) == str:
			v1 = values[self.t1]
		elif type(self.t1) == fn:
			v1 = self.t1.eval(values)
		else:
			raise ValueError(f"value of t1 ({self.t1} could not be determined")
#		print(f"t1 = {self.t1} ({type(self.t1)}) = {v1}")
		# return the result if the operator is unary
		if self.op in UNARY_OPS:
			return self.op(v1)
		# determine value of t2
		if type(self.t2) == int or type(self.t2) == float:
			v2 = self.t2 
		elif type(self.t2) == str:
			v2 = values[self.t2]
		elif type(self.t2) == fn:
			v2 = self.t2.eval(values)
		else:
			raise ValueError(f"value of t2 ({self.t2}) could not be determined")
#		print(f"t2 = {self.t2} ({type(self.t2)}) = {v2}")
		# return the result if the operator is binary
		return self.op(v1, v2)
	def __str__(self):
		if self.op in UNARY_OPS:
			return f"{{{UNARY_OPS[self.op][0]}{self.t1}{UNARY_OPS[self.op][1]}}}"
		else:
			return f"{{{self.t1}{BINARY_OPS[self.op]}{self.t2}}}"
	def __repr__(self):
		if self.op in UNARY_OPS:
			return f"{{{UNARY_OPS[self.op][0]}{self.t1}{UNARY_OPS[self.op][1]}}}"
		else:
			return f"{{{self.t1}{BINARY_OPS[self.op]}{self.t2}}}"


In [187]:
# an expression comprised of unprocessed text, values, and operations
class expr:
	def __init__(self, latex):
		# reduce an input equation into an input expression
		if "=" in latex:
			self.assign = latex[:latex.index("=")]
			self.expr = latex[latex.index("=")+1:]
		else:
			self.assign = None
			self.expr = latex
		# break out the brackets, ignoring latex display decorators for brackets ('\left' and '\right')
		self.ops = expr.parse(self.expr)
	# evaluate the expression
	def eval(self, values):
		return self.ops.eval(values)

	# parse the expression
	def parse(latex):
		# preprocessing
		text = latex.replace("\\left", "").replace("\\right", "")				# eliminate display-only tags
		text = text.replace("^", " ^ ").replace("-", " - ").replace("+", " + ")	# add spacing around operators to allow for easier detection
		while "  " in text:
			text = text.strip().replace("  ", " ")								# remove any double-spaces that got added
		
		# nest brackets
		struc = expr.nest(text)
		# refine the text into actual operations
		struc = expr.refine(struc)
		return struc

	# nest brackets in the expression
	def nest(latex, closure=""):
		text = latex.strip()	# trim any leading whitespace
		a = 0
		
		# nest the brackets
		structure = []
		while a < len(text):
			ch = text[a]
			# if the character is an opening bracket, consider only the remaining portion of the expression after the matching closing brace
			if ch in "([{":
				pt1 = text[:text.index(ch)].strip()
				pt2 = text[text.index(ch)+1:].strip()
				if pt1 != "":
					structure.append(pt1)
				contents, remainder = expr.nest(pt2, closure=CLOSING[ch])
				structure.append(contents)
				text = remainder.strip()
				a = 0
			# if the character closes the current set of brackets, return the two parts of the expression that were found
			elif ch==closure:
				pt1 = text[:text.index(ch)].strip()
				pt2 = text[text.index(ch)+1:].strip()
				if pt1 != "":
					for pt in pt1.split(" "):
						structure.append(pt)
				return structure, pt2
			# otherwise, move to the next letter
			else:
				a += 1
		if text != "":
			for pt in text.split(" "):
				structure.append(pt)
		# return the structure of the expression
		return structure

	# split up text
	def refine(struc):
		# anything in the 'fn' class has already been processed, so ignore it
		if type(struc) == fn:
			return struc
		# try converting strings (including single-item lists) to constants
		if type(struc) == str:
			try:
				return int(struc)
			except ValueError:
				try:
					return float(struc)
				except ValueError:
					return struc
		elif len(struc) == 1:
			return expr.refine(struc[0])
		# 2 item lists:
		elif len(struc) == 2:
			# check if the list is actually a subscripted variable that got broken up
			if type(struc[0]) == str and struc[0][-1] == "_" and type(struc[1])==list:
				return struc[0] + "".join(struc[1])
			# negative variables
			elif struc[0] == "-":
				return fn(op.neg, t1=expr.refine(struc[1]))
			# all other cases are implicitly multiplied together
			else:
				return fn(op.mul, t1=expr.refine(struc[0]), t2=expr.refine(struc[1]))
		# 3+ item lists
		else:
			# check for vertical division (based on brackets)
			if "\\frac" in struc:
				ind = struc.index("\\frac")
				return expr.refine(struc[:ind] + [fn(op.truediv, t1=expr.refine(struc[ind+1]), t2=expr.refine(struc[ind+2]))] + struc[ind+3:])
			# check for exponentiation
			elif "^" in struc:
				ind = struc.index("^")
				return expr.refine(struc[:ind-1] + [fn(op.pow, t1=expr.refine(struc[ind-1]), t2=expr.refine(struc[ind+1]))] + struc[ind+2:])
			# check for explicit multiplication/division
			elif "*" in struc or "×" in struc or "⋅" in struc or "/" in struc or "÷" in struc:
				ind = 0
				while struc[ind] not in ["*", "×", "⋅", "/", "÷"]:
					ind += 1
				return expr.refine(struc[:ind-1] + [fn(OPS[struc[ind]], t1=expr.refine(struc[ind-1]), t2=expr.refine(struc[ind+1]))] + struc[ind+2:])
			# the entire function must be checked for implicit multiplication before handling addition/subtraction
			else:
				ind = len(struc)-1
				while ind > 0:
					if struc[ind] not in ["+", "-"] and struc[ind-1] not in ["+", "-"]:
						struc[ind-1] = fn(op.mul, t1=expr.refine(struc[ind-1]), t2=expr.refine(struc[ind]))
						struc.pop(ind)
					ind -= 1
				# check for addition/subtraction
				if "+" in struc or "-" in struc:
					ind = 0
					while struc[ind] not in ["+", "-"]:
						ind += 1
					return expr.refine(struc[:ind-1] + [fn(OPS[struc[ind]], t1=expr.refine(struc[ind-1]), t2=expr.refine(struc[ind+1]))] + struc[ind+2:])
				else:
					return expr.refine(struc)
			




### Equation being considered:
$$P_E = \left(\frac{\pi}{k L}\right)^2 E I$$

In [188]:
# test case

code = fn(op.mul, fn(op.pow, fn(op.truediv, "pi", fn(op.mul, "k", "L")), 2), fn(op.mul, "E", "I"))
latex = r'P_E = \left(\frac{\pi}{k L}\right)^2 E I'
print(f"code:\t{code}")
print(f"latex:\t{latex}")
print("\n")

# calc(latex, {})
P_E = expr(latex)
input_values = {"\\pi":pi, "k":1.0, "L":3000*mm, "E":200*GPa, "I":0.918*10**6 * mm**4}
print(f"input values = {input_values}")
print(P_E.eval(input_values))


code:	{{{pi/{k×L}}^2}×{E×I}}
latex:	P_E = \left(\frac{\pi}{k L}\right)^2 E I


input values = {'\\pi': 3.141592653589793, 'k': 1.0, 'L': 1.000 m, 'E': 200.000 GPa, 'I': 918000.000 mm⁴}
1.812 MN
